In [2]:
import datasets


import tensorflow_datasets as tfds
config = tfds.translate.wmt.WmtConfig(
    version="0.0.1",
    language_pair=("fr", "de"),
    subsets={
        tfds.Split.TRAIN: ["commoncrawl_frde"],
        tfds.Split.VALIDATION: ["euelections_dev2019"],
    },
)
builder = tfds.builder("wmt_translate", config=config)

In [5]:
from datasets import load_dataset

dataset = load_dataset('wmt16', 'cs-en')

Dataset wmt16 downloaded and prepared to /home/jcxu/.cache/huggingface/datasets/wmt16/cs-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


In [10]:
test_set = dataset['test']
print(test_set[0])

{'translation': {'cs': 'Při nouzovém volání se profesor přiznal, že zastřelil svou přítelkyni', 'en': 'In 911 Call, Professor Admits to Shooting Girlfriend'}}


In [11]:
# T5 translation
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("t5-base")

model = AutoModelWithLMHead.from_pretrained("t5-base")

/home/jcxu/miniconda3/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:898: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [14]:
# facebook mbart
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
article_en = "The head of the United Nations says there is no military solution in Syria"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

model_inputs = tokenizer(article_en, return_tensors="pt")

# translate from English to Hindi
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => 'संयुक्त राष्ट्र के नेता कहते हैं कि सीरिया में कोई सैन्य समाधान नहीं है'

# translate from English to Chinese
generated_tokens = model.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
)

tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => '联合国首脑说,叙利亚没有军事解决办法'

['联合国首脑说,叙利亚没有军事解决办法']

In [20]:
print(generated_tokens)
print(tokenizer.decode([ 2, 250025,      6, 105699]))

tensor([[     2, 250025,      6, 105699,   6342,  38239,   1612,      4, 156446,
           3029,  63851,  14183,  30328,      2]])
</s>zh_CN 联合国


In [37]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."
article_zh = "全球主要经济体的领导人同意采取“有意义和有效的行动”，努力控制全球变暖。但是，在罗马举行的G20峰会上达成的协议几乎没有做出具体承诺，这让活动人士感到失望。主办国意大利曾希望在格拉斯哥召开的COP26峰会之前设定明确目标，而这次峰会现在已经开始了。"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")


In [44]:
print(tokenizer.special_tokens_map)
print(tokenizer.lang_code_to_id)
print(tokenizer.special_tokens_map['additional_special_tokens'])
# translate Hindi to English
tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The head of the UN says there is no military solution in Syria."
print(generated_tokens)
# print(tokenizer.lang_code_to_id("hi_IN"))
# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."
print(generated_tokens)


# translate Arabic to English
tokenizer.src_lang = "zh_CN"
encoded_ar = tokenizer(article_zh, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(generated_tokens)

# => "The Secretary-General of the United Nations says there is no military solution in Syria."



{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN', 'af_ZA', 'az_AZ', 'bn_IN', 'fa_IR', 'he_IL', 'hr_HR', 'id_ID', 'ka_GE', 'km_KH', 'mk_MK', 'ml_IN', 'mn_MN', 'mr_IN', 'pl_PL', 'ps_AF', 'pt_XX', 'sv_SE', 'sw_KE', 'ta_IN', 'te_IN', 'th_TH', 'tl_XX', 'uk_UA', 'ur_PK', 'xh_ZA', 'gl_ES', 'sl_SI']}
{'ar_AR': 250001, 'cs_CZ': 250002, 'de_DE': 250003, 'en_XX': 250004, 'es_XX': 250005, 'et_EE': 250006, 'fi_FI': 250007, 'fr_XX': 250008, 'gu_IN': 250009, 'hi_IN': 250010, 'it_IT': 250011, 'ja_XX': 250012, 'kk_KZ': 250013, 'ko_KR': 250014, 'lt_LT': 250015, 'lv_LV': 250016, 'my_MM': 250017, 'ne_NP': 250018, 'nl_XX': 250019, 'ro_RO': 250020, 'ru_RU': 

In [35]:

# translate Hindi to English
article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_zh="为什么是这？谁知道"
# translate Arabic to English
tokenizer.src_lang = "zh_CN"
encoded_ar = tokenizer(article_zh, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(generated_tokens)

print(tokenizer.decode([2,250010,44269],skip_special_tokens=False)) 
print(tokenizer.decode([2, 250025,6],skip_special_tokens=False))

tensor([[     2, 250022,  58353,   1816,     32,      2]])
</s>hi_IN संयुक्त
</s>zh_CN 


In [1]:
from datasets import load_metric
from sacrebleu.metrics import BLEU, CHRF, TER

In [7]:
bleu = BLEU()
sys = ['今天 是 个 好 日子']
refs=[['今天 天气 好', '今天 很 不错']]
print(bleu.corpus_score(sys, refs))
bleu.get_signature()
chrf = CHRF()

chrf.corpus_score(sys, refs)

BLEU = 12.70 40.0/12.5/8.3/6.2 (BP = 1.000 ratio = 1.667 hyp_len = 5 ref_len = 3)


chrF2 = 15.66

In [13]:
with open('/home/jcxu/.sacrebleu/wmt17/en-zh.zh', 'r') as fd:
    lines = fd.read().splitlines()
print(lines[:5])

# with open('/home/jcxu/.sacrebleu/wmt17/en-zh.en', 'r') as fd:
#     lines = fd.read().splitlines()
# print(lines[:5])

import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='zh')
md.detokenize(lines[0])

['28岁厨师被发现死于旧金山一家商场', '近日刚搬至旧金山的一位28岁厨师本周被发现死于当地一家商场的楼梯间。', '但受害人的哥哥表示想不出有谁会想要加害于他，并称“一切终于好起来了。”', '旧金山验尸官办公室表示，周三早上于西田购物中心发现的尸体确认为28岁旧金山居民 Frank Galicia。', '旧金山警察局称该起死亡案件被裁定为他杀，并正在进行调查。']


'2 8 岁厨师被发现死于旧金山一家商场'